In [22]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

def SNR(ground_truth, prediction):
    return np.mean(ground_truth ** 2) / np.mean((ground_truth - prediction) ** 2)


output_dir = "../../dataset/static/bunny/output/"
sub_dirs = glob(output_dir + "0.*")
sub_dirs = sorted(sub_dirs, key=lambda x: float(x.split("/")[-1]))
grid_size_list = [float(x.split("/")[-1]) for x in sub_dirs]
method_list = ['pppm', 'groundtruth', 'ghostcell1', 'ghostcell2']
for sub_dir, grid_size in zip(sub_dirs, grid_size_list):
    for method in method_list:
        ffat_file_list = glob(sub_dir + "/" + method + "/ffat*.txt")
        for ffat_file in ffat_file_list:
            data = np.loadtxt(ffat_file)
            ffat_size = int((len(data)/6)**0.5)
            ffat = data.reshape((6*ffat_size, ffat_size))
            # save the ffat as a png file
            plt.figure(figsize=(30, 5))
            plt.imshow((ffat / np.max(ffat)).T, cmap='gray')
            plt.savefig(ffat_file.replace(".txt", ".png"))
            # clear the figure
            plt.clf()
            plt.close()
